#### Model taken from:
- [1] Offshore Pipelaying Dynamics. Gullik Anthon Jensen 
- [2] A nonlinear PDE formulation for offshore vessel pipeline installation. Gullik A. Jensen et al 
- [3] Modeling and Control of Offshore Pipelay Operations Based on a Finite Strain Pipe Model. Gullik A. Jensen 

In [1]:
import numpy as np
import numdifftools as nd
import math

In [2]:
mp = 96 #  (submerged) [kg/m]

In [3]:
diag_Irho = 1e12*np.array([1, 1, 2]) # [m^4]  , p.99 in [1]
 
Irho=np.diag(diag_Irho)

In [4]:
Irho

array([[1.e+12, 0.e+00, 0.e+00],
       [0.e+00, 1.e+12, 0.e+00],
       [0.e+00, 0.e+00, 2.e+12]])

In [5]:
qw = 1025 # Water density [kg/m3]
d0 = 0.761 # Outer diameter of pipe, [m]

In [6]:
diag_DT = 1.5*np.array([1, 1, 1]) # [N/m]  , p.99 in [1]

In [7]:
DT=np.diag(diag_DT) # (35) in [2]

In [8]:
DT

array([[1.5, 0. , 0. ],
       [0. , 1.5, 0. ],
       [0. , 0. , 1.5]])

In [9]:
diag_CT = 1e9*np.array([1, 1, 1]) # p. 4721 in [3]

In [10]:
CT=np.diag(diag_CT)

In [11]:
from sympy import *

In [12]:
# Dphi1, Dphi2, Dphi3 = symbols("Dphi1 Dphi2 Dphi3")
dx,dy,dz=.3,4,5

In [13]:
def C1(dx,dy,dz):
    return np.array([dx**2,
          (dy**2+dz**2)**0.5*dy,
          (dy**2+dz**2)**0.5*dz]).T

In [14]:
DT

array([[1.5, 0. , 0. ],
       [0. , 1.5, 0. ],
       [0. , 0. , 1.5]])

In [15]:
def fD(dx,dy,dz,DT):
    return 1/2*d0*qw*np.dot(DT, C1(dx,dy,dz))

In [16]:
fD(dx,dy,dz,DT)

array([   52.6516875 , 14983.79094991, 18729.73868739])

In [17]:
# https://docs.scipy.org/doc/scipy-1.8.0/tutorial/integrate.html  
# 1-D Gray-Scott 

In [18]:
# Mat=np.matrix([[1.3,.4,6.7],[9,7,6],[3.3,5.6,7]])

In [19]:
I=np.identity(3)

In [20]:
fD(dx,dy,dz,DT)

array([   52.6516875 , 14983.79094991, 18729.73868739])

In [21]:
np.dot(I,fD(dx,dy,dz,DT))

array([   52.6516875 , 14983.79094991, 18729.73868739])

In [22]:
Rb_t=I # https://www.researchgate.net/profile/Thor-Fossen/publication/224560837_Modeling_and_Control_of_Offshore_Pipelay_Operations_Based_on_a_Finite_Strain_Pipe_Model/links/00b7d520e175a3f918000000/Modeling-and-Control-of-Offshore-Pipelay-Operations-Based-on-a-Finite-Strain-Pipe-Model.pdf

In [23]:
Rb_t

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [24]:
diag_DR = 1.5*np.array([1, 1, 1]) #   p.4721 in [3]

In [25]:
DR=np.diag(diag_DR) 

In [26]:
dI= 0.69# Inner diameter of pipe, [m]

In [27]:
A=math.pi*((d0/2)**2-(dI/2)**2)

In [28]:
fg_e=np.array([0,0,(mp-qw*A)*9.81])

In [29]:
from numpy import linalg as LA

In [30]:
def k_sigma(hi,d0,fg_e):
    if hi<0:
        return 0
    elif 0<=hi<=d0/20:
        return LA.norm(fg_e,2)/(d0/8-d0/40)*10*hi**2/d0
    else:
        return LA.norm(fg_e,2)/(d0/8-d0/40)*(hi-d0/40)

In [91]:
def hi(x,y,z,d0):
    return np.dot(np.array([x,y,z]).T,[0,0,1])+d0/2 

In [92]:
x,y,z=np.array([2.3,3.5,7])

In [93]:
def sigma(x,y,z):
    return k_sigma(hi(x,y,z,d0),d0,fg_e)*np.array([0,0,1])

In [94]:
# angles should be in radians, otherwise np.radians them  
def Re_t(φ,θ,ψ):
    Cφ=np.matrix([[1,0,0],
                  [0,np.cos(φ),-np.sin(φ)],
                  [0,np.sin(φ),np.cos(φ)]])
    
    Cθ=np.matrix([[np.cos(θ),0,np.sin(θ)],
                  [0,1,0],
                  [-np.sin(θ),0,np.cos(θ)]])
    
    Cψ=np.matrix([[np.cos(ψ),-np.sin(ψ),0],
                  [np.sin(ψ),np.cos(ψ),0],
                  [0,0,1]])
    
    return np.dot(np.dot(Cφ,Cθ),Cψ)

In [95]:
# vessel motion
# Fossen book p.384
# Fossen paper, section 6

In [96]:
# Fossen paper, equation (12)

In [97]:
φ,θ,ψ=0,1,2

In [98]:
def Re_b(φ,θ,ψ):
    return np.dot(Re_t(φ,θ,ψ), Rb_t ) # (5) in[3]

In [99]:
# p=np.dot(Re_b, phi) # (41) in[3]

In [100]:
def d_s(da,db,dc,dt,ds):
    return da*dt/ds,db*dt/ds,dc*dt/ds

In [101]:
x=[0,0.1]
y=[1,3]
z=[4,5]
ds=0.1

In [102]:
def ne(x,y,z,φ,θ,ψ,dt,ds):
    return np.dot(np.dot(np.dot(Re_t(φ,θ,ψ),CT),
                         Re_t(φ,θ,ψ).T), 
                  d_s(dx,dy,dz,dt,ds))

In [103]:
def ne_(x,y,z,dx,dy,dz):
    return -fg_e-fD(dx,dy,dz,DT)-sigma(x,y,z)

In [104]:
# mp*ddp = d_s(ne,ds)+np.dot(Re_b,ne_)

In [105]:
dt=.01

In [106]:
ne(x,y,z,φ,θ,ψ,dt,ds)

matrix([[2.5e+06, 4.0e+08, 5.0e+08]])

In [136]:
def grayscott1d(Q, t, ds):
    x,y,z=Q[0],Q[1],Q[2]
    dx,dy,dz=Q[3],Q[4],Q[5]
    φ,θ,ψ=Q[6],Q[7],Q[8]
    dφ,dθ,dψ=Q[9],Q[10],Q[11]
    
    C=ne(x,y,z,φ,θ,ψ,dt,ds)
    a,b,c=C[0,0],C[0,1],C[0,2]
    ddx,ddy, ddz = (1/mp*(d_s(a,b,c,dt,ds)
                                        +np.dot(Re_b(φ,θ,ψ),ne_(x,y,z,dx,dy,dz)))).reshape(3, 1) 
    
    return [dx, ddx, dy, ddy, dz, ddz, dφ, 0,dθ, 0,dψ,0]
    

In [137]:
q0=[0 for i in range(12)]
max_t = 10  
dt = 0.5
t=np.arange(0,max_t,dt)
dx = 0.025

In [138]:
from datetime import datetime
from scipy.integrate import odeint

In [139]:
startTime1 = datetime.now()
us=odeint(grayscott1d, q0, t, args=(ds,))
print(datetime.now() - startTime1)

0:00:00.568001


In [140]:
us.shape

(20, 12)

In [ ]:
def psi(gamma,omega,Ct,Cr):
    coord=np.concatenate((gamma, omega), axis=0)
    zero=np.zeros((3,3))
    M1=np.concatenate((Ct, zero), axis=0)
    M2=np.concatenate((zero, Cr), axis=0)
    M=np.concatenate((M1, M2), axis=1)
    return 1/2*np.dot(np.dot(coord.T,M),coord)

In [ ]:
from sympy import *

In [ ]:
gamma1, gamma2,gamma3 = symbols("gamma1 gamma2 gamma3")
omega1, omega2,omega3 = symbols("omega1 omega2 omega3")

In [ ]:
gamma=np.array([gamma1, gamma2,gamma3])
omega=np.array([omega1, omega2,omega3])

In [ ]:
Ct=np.matrix([[1,0,0],[0,2,0],[0,0,3]])
Cr=np.matrix([[4,0,0],[0,5,0],[0,0,6]])

In [ ]:
psi(gamma,omega,Ct,Cr)

In [ ]:
fun = lambda x, y: psi(x,y,Ct,Cr)

In [ ]:
Dfun = nd.Gradient(fun)

In [ ]:
Dfun()

In [ ]:
# coord=np.concatenate((gamma, omega), axis=0)

In [ ]:
gamma

In [ ]:
omega

In [ ]:
Dfun(gamma,omega)

In [ ]:
Dfun